In [4]:
import psycopg2
def get_db_connection():
    try:
        conn = psycopg2.connect(
            host="172.27.80.1",
            database="calpuff",
            user="postgres",
            password="1201"
        )
        print("데이터베이스에 연결되었습니다.")
        return conn
    except Exception as e:
        print("데이터베이스 연결에 실패했습니다:", e)
        return None

## URL 저장 및 조회

In [31]:
import requests
from bs4 import BeautifulSoup
import psycopg2
import time
from datetime import datetime
import asyncio
import aiohttp
from tqdm import tqdm
import nest_asyncio

from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [28]:
import asyncio

def check_running_loop():
    try:
        loop = asyncio.get_running_loop()
        print("현재 실행 중인 이벤트 루프가 있습니다:", loop)
    except RuntimeError:
        print("현재 실행 중인 이벤트 루프가 없습니다.")

# 테스트
check_running_loop()

현재 실행 중인 이벤트 루프가 있습니다: <_UnixSelectorEventLoop running=True closed=False debug=False>


In [ ]:
# tbody 안의 모든 tr 태그 찾기

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

chrome_options = Options()

driver = webdriver.Chrome(options=chrome_options)

url = 'https://rda.ucar.edu/datasets/d083002/dataaccess/'

try:
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    button1 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#ds_content > div.mtrx.mx-1 > div:nth-child(32) > a')))
    actions1 = ActionChains(driver).move_to_element(button1)
    actions1.perform()
    button1.click()
    
    time.sleep(3)
    
    button2 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#ds_content > div > div > div > div:nth-child(3) > div > div > a')))
    actions2 = ActionChains(driver).move_to_element(button2)
    actions2.perform()
    button2.click()
    
    time.sleep(3)
    
    button3 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#none_table > tbody > tr:nth-child(18) > td.Group.Name > a')))
    actions3 = ActionChains(driver).move_to_element(button3)
    actions3.perform()
    button3.click()
    
    time.sleep(60)
    
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
        
    rows = soup.select("#grib2-202401_table > tbody > tr")
    if not rows:
        print('tr 태그를 찾을 수 없습니다.')
    else:
        for idx, row in enumerate(rows, start=1):
            a_tag = row.select_one("td.File.Name > a:nth-child(1)")
            if a_tag:
                print(f"{idx}번째 항목에서 파일 링크를 찾았습니다: {a_tag['href']}")
            else:
                print(f"{idx}번째 항목에서 파일 링크를 찾을 수 없습니다.")

except Exception as e:
        print("오류 발생:", e)
        
finally:
    driver.quit()


전체 코드

In [34]:
nest_asyncio.apply()

success_count = 0
fail_count = 0

# selenium 설정 및 파일 링크 추출하기
async def extract_file_links():
    loop = asyncio.get_running_loop()
    with ThreadPoolExecutor() as pool:
        links = await loop.run_in_executor(pool, sync_extract_file_links)
    return links
    
def sync_extract_file_links():
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    url = 'https://rda.ucar.edu/datasets/d083002/dataaccess/'
    
    links = []
    
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        
        button1 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#ds_content > div.mtrx.mx-1 > div:nth-child(32) > a')))
        ActionChains(driver).move_to_element(button1).click().perform()
        
        time.sleep(3)
        
        button2 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#ds_content > div > div > div > div:nth-child(3) > div > div > a')))
        ActionChains(driver).move_to_element(button2).click().perform()
        
        time.sleep(3)
        
        button3 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#none_table > tbody > tr:nth-child(18) > td.Group.Name > a')))
        ActionChains(driver).move_to_element(button3).click().perform()
        
        time.sleep(60)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        rows = soup.select("#grib2-202401_table > tbody > tr")

        for idx, row in enumerate(rows, start=1):
            a_tag = row.select_one("td.File.Name > a:nth-child(1)")
            if a_tag:
                file_name = a_tag.get_text(strip = True)
                href = a_tag['href']
                links.append([file_name, href])
                print(f"{idx}번째 항목에서 파일 링크를 찾았습니다: {href}")
            else:
                print(f"{idx}번째 항목에서 파일 링크를 찾을 수 없습니다.")

    except Exception as e:
        print("오류 발생:", e)
        return []
    finally:
        driver.quit()
        
    return links

# 데이터 베이스에 넣기
async def insert_data_if_new(session, file_name, url_path):
    global success_count, fail_count

    # 데이터베이스 연결
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        time_value = file_name.split('_')[1] + '_' + file_name.split('_')[2] + '_' + file_name.split('_')[3].split('.')[0]
    
        # 중복 확인
        cursor.execute("SELECT * FROM NCAR_data WHERE time = %s", (time_value,))
        existing_data = cursor.fetchone()
        
        if existing_data:
            print(f'{file_name}: 중복된 데이터입니다. 삽입을 건너뜁니다.')
            fail_count += 1
            return
        
        # 새 데이터 삽입
        cursor.execute("INSERT INTO NCAR_data (time, file_name, url_path) VALUES (%s, %s, %s)", 
                        (time_value, file_name, url_path))
        conn.commit()
        print("새로운 데이터가 추가되었습니다:", file_name)
        success_count += 1
        
    except Exception as e:
        print('데이터 처리 중 오류 발생: ', e)
        fail_count += 1
    finally:
        cursor.close()
        conn.close()

# async def monitor_db():
#     """데이터 삽입 진행 중 실시간으로 DB 상태를 모니터링합니다."""
#     conn = get_db_connection()
#     cursor = conn.cursor()
#     last_checked_count = 0

#     try:
#         while True:
#             cursor.execute("SELECT COUNT(*) FROM NCAR_data")
#             row_count = cursor.fetchone()[0]

#             # 새로운 데이터가 추가되었는지 확인
#             if row_count > last_checked_count:
#                 print(f"새로운 데이터가 {row_count - last_checked_count}개 추가되었습니다. 총 {row_count}개의 데이터가 있습니다.")
#                 last_checked_count = row_count
            
#             await asyncio.sleep(2)  # 2초마다 데이터베이스 상태 확인
#     except Exception as e:
#         print("모니터링 중 오류 발생:", e)
#     finally:
#         cursor.close()
#         conn.close()

async def main():
    global success_count, fail_count
    success_count = 0
    fail_count = 0

    print('URL 불러오기 시작 ..')
    
    links = await extract_file_links()

    
    async with aiohttp.ClientSession() as session:
        insert_tasks = [insert_data_if_new(session, file_name, url_path) for file_name, url_path in links]
        # monitor_task = monitor_db()  # DB 상태 모니터링 작업 추가
        
        await asyncio.gather(*insert_tasks)  # 삽입 작업과 모니터링 작업 동시에 실행
        # await asyncio.gather(*insert_tasks, monitor_task)  # 삽입 작업과 모니터링 작업 동시에 실행
        

    print('URL 불러오기 완료')
    print(f'성공: {success_count}')
    print(f'실패: {fail_count}')

# 실행
await main()  # Jupyter Notebook이나 VS Code에서 실행

URL 불러오기 시작 ..


/home/yurim2/anaconda3/envs/venv-CALPUFF/lib/python3.8/site-packages/bs4/__init__.py:749: RuntimeWarning: coroutine 'extract_file_links' was never awaited
  tag = self.element_classes.get(Tag, Tag)(


1번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_20240101_00_00.grib2 
2번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_20240101_06_00.grib2 
3번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_20240101_12_00.grib2 
4번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_20240101_18_00.grib2 
5번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_20240102_00_00.grib2 
6번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_20240102_06_00.grib2 
7번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_20240102_12_00.grib2 
8번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_20240102_18_00.grib2 
9번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_20240103_00_00.grib2 
10번째 항목에서 파일 링크를 찾았습니다: https://data.rda.ucar.edu/d083002/grib2/2024/2024.01/fnl_2